<a href="https://colab.research.google.com/github/zahraDehghanian97/ddcrp_Cora/blob/master/ddcrp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install torch_geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 23.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910476 sha256=2932a3c4e877f09f28f36c88e65914cb43b5d390d91abbe6cd0649c229f769f5
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch_geometric


In [6]:
from torch_geometric.datasets import Planetoid
dataset = Planetoid("/content/sample_data/Cora" , name = 'Cora')


Processing...
Done!


In [32]:
import numpy as np
from numpy import seterr, isneginf, array
from scipy.special import gammaln
from scipy.special import logsumexp

def dirichlet_likelihood(Xp, hyper):
    if len(Xp.shape) == 2:
        X =sum(Xp)
    else:
        X = Xp
    idx = np.where(X!=0)
    lh = gammaln(len(X)*hyper) + sum(gammaln(X[idx]+hyper))\
    -len(idx)*gammaln(hyper)  - gammaln(sum(X)+len(X) * hyper)
    return lh

def linear_distance(i,j):
    return i-j

def window_delay(a,size=1):
    if abs(a) <= size and a >= 0:
        return 1;
    else:
        return 0;

#get the customers linked to customer i
def get_linked(i,link):
    c = []
    q = []
    q.append(i)
    while q:
        cur = q[0]
        c.append(cur)
        for k in range(0,len(link)):
            if (link[k] == cur) and (k not in c) and (k not in q):
                q.append(k)
        q = q[1:]
    return c

def ddcrp_infer(obs,lhood_fn,distance,delay,n_iter,alpha = 0.2):
    n = len(obs)
    cluster = np.array([0]*n)
    link = np.array([0]*n)
    prior = np.random.random(n*n).reshape((n,n))
    #lhood = np.random.random(n)
    merged_lhood = np.random.random(n)
    lhood = list(map(lambda x: lhood_fn(obs[np.where(cluster == x)]) , cluster))  #lhood of each cluster

    obs_lhood = 0 #the likelihood of all obs

    #prior of each customer
    for i in range(0,n):
        for j in range(0,n):
            try:
                if i==j:
                    prior[i][j] = np.log(alpha)
                else:
                    seterr(divide='ignore')
                    prior[i][j] = np.log(delay(distance(i,j)))
                    seterr(divide='warn')
                    prior[i][j][isneginf(prior[i][j])] = 0
            except Exception as e:
                # print(e)
                pass


    for t in range(0,n_iter):
        print("iter "+str(t))
        obs_lhood = 0
        for i in range(0,n):
            #print "sample"+str(i)+"th:"
            #remove the ith's link
            old_link = link[i]
            old_cluster = cluster[old_link]
            cluster[i] = i
            link[i] = i
            linked = get_linked(i,link)
            # print(linked)
            cluster[linked] = i

            if old_cluster not in linked :
                idx = np.where(cluster == old_cluster)
                lhood[old_cluster] = lhood_fn(obs[idx])
                lhood[i] = lhood_fn(obs[linked])


            #calculate the likelihood of the merged cluster
            for j in np.unique(cluster):

                if j == cluster[i] :
                    merged_lhood[j] = 2*lhood_fn(obs[linked])
                else:
                    merged_lhood[j] = lhood_fn(np.concatenate((obs[linked] , obs[np.where(cluster == j)])))

            log_prob = list(map(lambda x: prior[i][x] + merged_lhood[cluster[x]] - lhood[cluster[x]]-lhood[cluster[i]], np.arange(n)))
            prob = np.exp(log_prob - logsumexp(log_prob))

            #sample z_i
            link[i] = np.random.choice(np.arange(n),1,p=prob)

            #update the likelihood if the link sample merge two cluster
            new_cluster = cluster[link[i]]
            if new_cluster !=i:
                cluster[linked] = new_cluster
                lhood[new_cluster] = merged_lhood[new_cluster]

            #cal the likelihood of all obs
            for u in np.unique(cluster):
                obs_lhood = obs_lhood + lhood[u]

        print("cluster")
        print(cluster)
        print("link")
        print(link)
        print(obs_lhood)

    return cluster,link,obs_lhood

In [24]:
print(dataset.data.keys)
features = dataset.data.x
x = features.detach().cpu().numpy()[0:100]
print(x.shape)

['x', 'edge_index', 'train_mask', 'test_mask', 'val_mask', 'y']
(100, 1433)


In [31]:

# x = np.array([[10,0,0,0,0],[10,0,0,0,0],[5,0,0,0,0],[11,0,0,0,1],[0,10,0,0,1],[0,10,0,0,0],[0,0,10,0,0],[0,1,10,0,0],[20,0,2,0,0],[10,0,0,1,0],[10,1,0,10,0],[10,0,2,10,0],[10,0,0,10,0],[10,1,0,1,0],[10,0,0,0,0],])

#initalize parameters
n_iter =20 #200
hyper = 0.01
alpha = 0.01
window_size = 20 #20

#bookkeeper data
#cluster,link,obs,lhood,
lhood_fn = lambda x:dirichlet_likelihood(x,hyper)
distance = linear_distance
delay = lambda x:window_delay(x,window_size)

cluster,link,lhood = ddcrp_infer(x,lhood_fn,distance,delay,n_iter,alpha)

iter 0
cluster
[ 0  1  1  3  3  1  6  6  8  3  8 11  8 13 14 13 16 16 16  6 20 20 22 11
 24 22 26 24 14 13 30 31 30 26 34 31 34 37 38 39 39 38 42 42 37 45 45 45
 48 49 48 51 34 53 34 49 53 57 49 51 60 60 57 63 64 65 34 63 68 64 70 70
 68 68 74 74 76 77 65 77 76 81 81 83 84 83 81 84 88 89 90 90 92 88 88 95
 96 96 98 98]
link
[ 0  1  1  3  3  1  6  6  8  3  8 11 10 13 14 13 16 16 17  7 20 20 22 11
 24 22 26 24 14 13 30 31 30 26 34 31 34 37 38 39 39 38 42 42 37 45 45 45
 48 49 48 51 36 53 52 49 53 57 55 51 60 60 57 63 64 65 54 63 68 64 70 70
 68 72 74 74 76 77 65 77 76 81 81 83 84 83 82 84 88 89 90 90 92 88 93 95
 96 96 98 98]
-859426.3796952959
iter 1
cluster
[ 0  0  2  3  4  2  6  7  8  4 10  6  7 13 14  8 16 10  3  7 20 20 22  2
 24 13 26 22  4 10 30 31 14 24 34 16 36 37 38 39 40 34 42 40 38 45 37 37
 48 49 30 51 36 53 45 55 53 57 49 55 60 61 48 63 64 65 61 36 68 51 70 45
 70 63 74 75 76 77 78 68 61 81 64 83 84 65 75 83 88 89 90 74 92 88 92 77
 96 90 98 98]
link
[ 0  0  2  3  4  2  6  